<a href="https://colab.research.google.com/github/arindamcodes/ToyReasoningModels/blob/main/reasoning_model_grpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import subprocess
import tempfile
import os

In [3]:
device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')

In [4]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
prompt = "what is 1 + 1"

input_ids = tokenizer(prompt, return_tensors="pt").to(device)

output_ids = model.generate(input_ids['input_ids'], max_new_tokens=64, do_sample=True, temperature=0.7, pad_token_id=tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [6]:
input_ids

{'input_ids': tensor([[    1,   825,   338, 29871, 29896,   718, 29871, 29896]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
output_ids

tensor([[    1,   825,   338, 29871, 29896,   718, 29871, 29896,    13,    13,
         29902, 29915, 29885, 24173,   304,  5110,   716,  2712,   322,  5503,
           590, 13071, 29889,     2]])

In [8]:
decoded_text = tokenizer.decode(output_ids[0][input_ids['input_ids'].shape[1]:], skip_special_tokens=True)
print(decoded_text)



I'm excited to learn new things and push my limits.


In [9]:
logits = model(output_ids[:,:-1])


In [10]:
logits.logits

tensor([[[ -4.6445,   1.0234,   4.5312,  ...,  -5.1836,  -2.1113,  -4.2070],
         [-10.7344, -10.6875,   1.0879,  ...,  -7.6250,  -8.3203,  -7.0273],
         [ -9.1406,  -9.2656,   1.9219,  ...,  -5.9023,  -9.2188,  -6.6719],
         ...,
         [ -8.6562,  -8.3984,   4.0508,  ...,  -5.9531,  -5.3594,  -5.4961],
         [ -8.3516,  -7.7734,   9.4141,  ...,  -5.1562,  -5.8594,  -4.0820],
         [ -7.4922,  -6.7109,  14.4375,  ...,  -2.5684,  -6.7070,  -3.2500]]],
       dtype=torch.float16, grad_fn=<UnsafeViewBackward0>)

In [11]:
logp = F.log_softmax(logits.logits, dim=-1)
logp

tensor([[[-16.3594, -10.6953,  -7.1875,  ..., -16.9062, -13.8281, -15.9219],
         [-23.1562, -23.1094, -11.3359,  ..., -20.0469, -20.7500, -19.4531],
         [-21.8125, -21.9375, -10.7578,  ..., -18.5781, -21.8906, -19.3438],
         ...,
         [-23.8281, -23.5625, -11.1250,  ..., -21.1250, -20.5312, -20.6719],
         [-25.2656, -24.6875,  -7.5000,  ..., -22.0625, -22.7812, -21.0000],
         [-23.0781, -22.2969,  -1.1533,  ..., -18.1562, -22.2969, -18.8438]]],
       dtype=torch.float16, grad_fn=<LogSoftmaxBackward0>)

In [12]:
logp.shape

torch.Size([1, 23, 32000])

In [13]:
batch_size, T = output_ids.shape[0], output_ids.shape[1]-1
gen_logp = torch.empty(batch_size, T)

for b in range(batch_size):
    for t in range(T):
        true_id = output_ids[b, t+1]
        gen_logp[b, t] = logp[b, t, true_id]

In [14]:
len(gen_logp)

1

In [15]:
gen_logp[0]

tensor([-10.4688,  -3.2109,  -6.5625,  -1.8584,  -4.0938,  -0.1204,  -0.8833,
         -3.2344,  -1.6689,  -3.8711,  -2.3516,  -0.4705,  -4.5547,  -0.1979,
         -1.9521,  -2.9648,  -1.0010,  -0.5488,  -4.6289,  -0.8394,  -1.4766,
         -1.1016,  -1.1533], grad_fn=<SelectBackward0>)

In [16]:
next_ids   = output_ids[:, 1:]
print(next_ids[0, 2])
print(logp[0, 2, next_ids[0, 2]])

tensor(29871)
tensor(-6.5625, dtype=torch.float16, grad_fn=<SelectBackward0>)


In [17]:
model.train()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

In [19]:
# We will fine tune the base model for basic coding task as toy examples

# Reward function in case of basic coding problem is whether it matches the expected output or not for given no of test cases
def compute_reward(code_str, test_cases):

  '''
  if the excuted python code is able to match the output then we will reward the model as 1 or else 0
  then out of total test cases we will check how many are getting matched and put that as final reward for that sequence of actions
  '''

  with tempfile.TemporaryDirectory() as tmpdir:
    script_path = os.path.join(tmpdir, "script.py")
    with open(script_path, "w") as f:
      f.write(code_str)


  passed = 0
  for case in test_cases:
    try:
      proc = subprocess.run(
          ["python3", script_path],
          input=case["input"],
          text=True,
          capture_output=True,
          timeout=2
      )

      output = proc.stdout.strip()
      if output == case["output"]:
        passed += 1
    except Exception:
      continue
    return passed / len(test_cases) if test_cases else 0.0


In [24]:
# Next we are going to implement the GRPO according to the deepseek paper. In case of any doubt please consult the paper once
def grpo_epoch(samples, model, tokenizer, optimizer=None,
               group_size=3, device="cpu"):


    total_loss, total_rewards, count = 0.0, 0.0, 0

    for sample in samples:
      prompt = sample["prompt"]
      test_cases = sample["tests"]

      input_enc = tokenizer(prompt, return_tensors="pt").to(device)
      input_ids = input_enc["input_ids"]
      codes, rewards, log_probs = [], [], []


      for _ in range(group_size):
        with torch.no_grad():
          output_ids = model.generate(
              input_ids,
              max_new_tokens=64,
              do_sample=True,
              temperature=0.7,
              pad_token_id=tokenizer.eos_token_id
          )


        gen_tokens = output_ids[0, input_ids.shape[1]:]
        code = tokenizer.decode(gen_tokens, skip_special_tokens=True)
        codes.append(code)


        reward = compute_reward(code, test_cases)
        rewards.append(reward)

        outputs = model(output_ids)
        logits = outputs.logits
        logp = F.log_softmax(logits, dim=-1)


        start = input_ids.shape[1]
        '''
        Simplied version of the code:
        batch_size, seq_len = output_ids.shape

        gen_len = seq_len - input_ids.shape[1]  # or seq_len - start

        gen_logp = torch.empty(batch_size, gen_len, device=logp.device, dtype=logp.dtype)

        for b in range(batch_size):

            for t in range(gen_len):

                true_id = output_ids[b, input_ids.shape[1] + t]
                gen_logp[b, t] = logp[b, input_ids.shape[1] + t, true_id]

        seq_logp = gen_logp.sum(dim=1)
        log_probs.append(seq_logp)
        '''
        gen_logp = logp[0, start:-1, :].gather(
            dim=-1,
            index=output_ids[0, start+1:].unsqueeze(-1)
        ).squeeze(-1)  # vectorized version of the above implementation

        seq_logp = gen_logp.sum()
        log_probs.append(seq_logp)


      rewards = torch.tensor(rewards, dtype=torch.float32, device=device)
      log_probs = torch.stack(log_probs)


      mean_r = rewards.mean()
      std_r = rewards.std(unbiased=False) + 1e-8

      advantages = (rewards - mean_r) / std_r


      loss = -(advantages * log_probs).mean()

      if optimizer is not None:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


      total_loss += loss.item()
      total_rewards += rewards.sum().item()
      count += 1

    avg_loss = total_loss / count
    avg_reward = total_rewards / count

    return avg_loss, avg_reward

In [25]:
# ── TRAINING SET ────────────────────────────────────────────────────────────────
toy_train = [
    {
        "prompt": "def add(a, b):\n    # return the sum of a and b\n",
        "tests": [
            {"input": "2 3\n", "output": "5"},
            {"input": "10 15\n", "output": "25"},
            {"input": "0 0\n", "output": "0"},
        ],
    },
    {
        "prompt": "def factorial(n):\n    # compute n!\n",
        "tests": [
            {"input": "0\n", "output": "1"},
            {"input": "4\n", "output": "24"},
            {"input": "6\n", "output": "720"},
        ],
    },
    {
        "prompt": "def is_prime(n):\n    # print 'yes' if n is prime else 'no'\n",
        "tests": [
            {"input": "2\n", "output": "yes"},
            {"input": "9\n", "output": "no"},
            {"input": "17\n", "output": "yes"},
        ],
    },
]

# ── VALIDATION SET ─────────────────────────────────────────────────────────────
toy_val = [
    {
        "prompt": "def subtract(a, b):\n    # return a minus b\n",
        "tests": [
            {"input": "5 3\n", "output": "2"},
            {"input": "0 7\n", "output": "-7"},
        ],
    },
    {
        "prompt": "def fib(n):\n    # print the n-th Fibonacci number\n",
        "tests": [
            {"input": "1\n", "output": "1"},
            {"input": "5\n", "output": "5"},
        ],
    },
]

# ── TEST SET ──────────────────────────────────────────────────────────────────
toy_test = [
    {
        "prompt": "def multiply(a, b):\n    # return a * b\n",
        "tests": [
            {"input": "3 4\n", "output": "12"},
            {"input": "7 0\n", "output": "0"},
        ],
    },
    {
        "prompt": "def reverse_str(s):\n    # print the reverse of the input string\n",
        "tests": [
            {"input": "hello\n", "output": "olleh"},
            {"input": "racecar\n", "output": "racecar"},
        ],
    },
]

In [26]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
num_epochs = 3

In [ ]:
for epoch in range(num_epochs):
    train_loss, train_reward = grpo_epoch(toy_train, model, tokenizer, optimizer, device=device)
    val_loss, val_reward = grpo_epoch(toy_val, model, tokenizer, optimizer=None, device=device)
    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Train Reward: {train_reward:.3f} | "
          f"Val Loss: {val_loss:.4f} | Val Reward: {val_reward:.3f}")